<img src="https://raw.githubusercontent.com/OpenEnergyPlatform/academy/develop/docs/data/img/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="75" width="75" align="left"/>

# Upload csv data to a new table on the OEP using the oedialect

Repository: https://github.com/openego/oedialect <br>
Documentation: http://oep-data-interface.readthedocs.io/en/latest/api/how_to.html

Please report bugs and improvements here: https://github.com/OpenEnergyPlatform/oedialect/issues <br>

In [1]:
__copyright__ = "Reiner Lemoine Institut"
__license__   = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__       = "https://github.com/openego/data_processing/blob/master/LICENSE"
__author__    = "christian-rli, oakca"

In [2]:
# pip install oedialect pandas "sqlalchemy~=1.3,<1.4"

import os
import getpass

import pandas as pd
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
import oedialect


## Reading data from a csv file and uploading it to the oedb

Pandas has a read_csv function which makes importing a csv-file rather comfortable. It reads csv into a DataFrame. By default, it assumes that the fields are comma-separated. Our example file has columns with semicolons as separators, so we have to specify this when reading the file. 

The example file for this tutorial ('DataTemplate.csv') is in a 'data' directory, one level above the file for this tutorial. Make sure to adapt the path to the file you're using if your file is located elsewhere.


In [3]:
example_df = pd.read_csv('../../data/TemplateData.csv', encoding='utf8', sep=';')

Looking at the first three lines of our dataframe:

In [4]:
example_df[:3]

,variable,unit,year,value
0,Antimatter price,Euro (2010) / GJ,2020,3.00
1,Antimatter price,Euro (2010) / GJ,2030,3.30
2,Antimatter price,Euro (2010) / GJ,2040,3.33


## Connection to OEP

If we want to upload data to the OEP we first need to connect to it, using our OEP user name and token. 
Note: You ca view your token on your OEP profile page after logging in.  

In [5]:
# NEVER commit your token to a repository
# get your token from an environment variable
# or ask user
token = os.environ.get("OEP_API_TOKEN") or getpass.getpass('Token:')

Now we'll create an sql-alchemy-engine. The engine is what 'speaks' oedialect to the data base api. We need to tell it where the data base is and pass our credentials.


In [6]:
# Create Engine:
oep_url = 'openenergy-platform.org'
oed_string = f'postgresql+oedialect://:{token}@{oep_url}'

engine = sa.create_engine(oed_string)
metadata = sa.MetaData(bind=engine)
print(metadata)

MetaData(bind=Engine(postgresql+oedialect://:***@openenergy-platform.org))


## Setup a Table

We need to tell the data base what columns and datatypes we are about to upload. In our case we have four columns, two of which are text, one is integer and the last is float.

In [7]:
table_name = 'example_dialect_tablexon'
schema_name = 'sandbox'

ExampleTable = sa.Table(
    table_name,
    metadata,
    sa.Column('id', sa.INTEGER),
    sa.Column('variable', sa.VARCHAR(50)),
    sa.Column('unit', sa.VARCHAR(50)),
    sa.Column('year', sa.INTEGER),
    sa.Column('value', sa.FLOAT(50)),
    schema=schema_name
)

## Create the new Table

Now we tell our engine to connect to the data base and create the defined table within the chosen schema.

In [8]:
conn = engine.connect()
print('Connection established')
if not engine.dialect.has_table(conn, table_name, schema_name):
    ExampleTable.create()
    print('Created table')
else:
    print('Table already exists')

Connection established
Created table


## Insert data into Table
 
Uploading the information from our DataFrame is now done with a single command. Uploading data in this way will always delete the content of the table and refill it with new values every time. If you change 'replace' to 'append', the data entries will be added to the preexisting ones. (Connecting and uploading may take a minute.)

In [9]:
Session = sessionmaker(bind=engine)
session = Session()
try: 
    insert_statement = ExampleTable.insert().values(example_df.to_dict(orient='records'))
    session.execute(insert_statement)
    session.commit()
    print('Inserted to ' + table_name)
except Exception as e:
    session.rollback()    
    print('Insert incomplete!')
    raise
finally:
    session.close()

Inserted to example_dialect_tablexon


You can also insert data manually into the table.

In [10]:
Session = sessionmaker(bind=engine)
session = Session()
try:
    insert_statement = ExampleTable.insert().values(
        [
            dict(variable='fairy dust', unit='t', year=2020, value=200),
            dict(variable='mana', unit='kg', year=1999, value=120),
            dict(variable='the force', unit='l', year=1998, value=1100)
        ]
    )
    session.execute(insert_statement)
    session.commit()
    print('Insert successful!')
except Exception as e:
    session.rollback()    
    print('Insert incomplete!')
    raise
finally:
    session.close()

Insert successful!


## Select from Table

Now  we can query our table to see if the data arrived.

In [11]:
Session = sessionmaker(bind=engine)
session = Session()
print(session.query(ExampleTable).all())
session.close()

[(1, 'Antimatter price', 'Euro (2010) / GJ', 2020, 3.0), (2, 'Antimatter price', 'Euro (2010) / GJ', 2030, 3.3), (3, 'Antimatter price', 'Euro (2010) / GJ', 2040, 3.33), (4, 'Antimatter price', 'Euro (2010) / GJ', 2050, 4.44), (5, 'Fusion price', 'Euro (2010) / GJ', 2020, 1.7), (6, 'Fusion price', 'Euro (2010) / GJ', 2030, 1.7), (7, 'Fusion price', 'Euro (2010) / GJ', 2040, 1.7), (8, 'Fusion price', 'Euro (2010) / GJ', 2050, 1.7), (9, 'Naquadria price', 'Euro (2010) / GJ', 2020, 10.0), (10, 'Naquadria price', 'Euro (2010) / GJ', 2030, 18.1), (11, 'Naquadria price', 'Euro (2010) / GJ', 2040, 26.2), (12, 'Naquadria price', 'Euro (2010) / GJ', 2050, 32.3), (13, 'Power Cristals price', 'Euro (2010) / GJ', 2020, 5.5), (14, 'Power Cristals price', 'Euro (2010) / GJ', 2030, 9.9), (15, 'Power Cristals price', 'Euro (2010) / GJ', 2040, 11.11), (16, 'Power Cristals price', 'Euro (2010) / GJ', 2050, 14.4), (17, 'CO2 certificate price ', 'Euro (2010) / ton', 2020, 40.0), (18, 'CO2 certificate pric

## Storing Query Result in DataFrame
We can write the results of the query back into a DataFrame, where it's easier to handle.

In [12]:
Session = sessionmaker(bind=engine)
session = Session()
df = pd.DataFrame(session.query(ExampleTable).all())
session.close()
df

,id,variable,unit,year,value
0,1,Antimatter price,Euro (2010) / GJ,2020,3.00
1,2,Antimatter price,Euro (2010) / GJ,2030,3.30
2,3,Antimatter price,Euro (2010) / GJ,2040,3.33
3,4,Antimatter price,Euro (2010) / GJ,2050,4.44
4,5,Fusion price,Euro (2010) / GJ,2020,1.70
5,6,Fusion price,Euro (2010) / GJ,2030,1.70
6,7,Fusion price,Euro (2010) / GJ,2040,1.70
7,8,Fusion price,Euro (2010) / GJ,2050,1.70
8,9,Naquadria price,Euro (2010) / GJ,2020,10.00
9,10,Naquadria price,Euro (2010) / GJ,2030,18.10


# Delete Table

In [13]:
ExampleTable.drop()